In [2]:
import pandas as pd
import numpy as np 
#from faker import Faker
import re, json, logging , html, os
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
#from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.chrome.options import Options
#from selenium import webdriver
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup   
#Faker.seed(0)
#fake = Faker()
#import nltk
#nltk.download("punkt")
#from nltk.corpus import stopwords 
#sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [4]:
ip = issue_provider(ctx)    
for i in range(9596,9605,1):
    df=ip.parse_metrics(str(i), parse_to=f'ig\{i}.csv', met_parse="(\d{1,2}\.\d?\.?)\s{1,5}[A-Z]", usecache=True) 
ip.driver.quit()



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST driver version for 113.0.5672
Driver [C:\Users\timko\.wdm\drivers\chromedriver\win32\113.0.5672.63\chromedriver.exe] found in cache


In [ ]:
ip = issue_provider(ctx)    
core=[]
reqsup=[]
for i in range(9596,9605,1): #9605
    df=ip.parse_metrics(str(i), parse_to=f'ig\{i}.csv', met_parse="(\d{1,2}\.\d?\.?)\s{1,5}[A-Z]") 
    dff=df
    for i,r in dff.iterrows():
        elm = str(r['element'])[:50]
        if '<em>' in elm:
            reqsup.append(str(r['{idt}']))
        if '<b>' in elm:
            core.append(str(r['{idt}'])) 
ip.driver.quit()

In [ ]:
s="'"+''.join(reqsup).replace('.','\',\'')[:-2]

c="'"+''.join(core).replace('.','\',\'')[:-2] 
print(s) 


In [ ]:
df, code = generate_code_from_db(ctx, 4452)  
with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw:
    fw.write(code)

In [ ]:
def generate_pages(PK_FORM='2023-A-HVA', classname=None, where=None, commit=False, PageNameFilter='~', dest=None ):
    sql= f"""
        SELECT DISTINCT PK_FORM, PK_QuestionGroup, SectionNum, GroupName, ASPX
        FROM vwQuestions  WHERE 1=1 AND PK_FORM='{PK_FORM}' 
        --AND {where}
        ORDER BY PK_QuestionGroup ASC
    """
    if classname == None:
        classname=PK_FORM.replace('-','_')
    if where != None:
        sql=sql.replace('--AND', 'AND  ')
    #print(sql)
    df=sql_todf(sql, connstr)     
    df['PNAME']=df['ASPX'].apply(lambda s: s.split('/')[len(s.split('/'))-1].replace('.aspx',''))
    pages=df.to_dict(orient='records')    
    src=ctx.get_tempalte_dir()
    out=''
    for p in pages:   
        PageName = re.sub(PageNameFilter,'',f"{classname}_{p['GroupName']}")
        for ext in ['.aspx','.aspx.designer.vb','.aspx.vb']:
            with open(f'{src}{classname}{ext}', 'r', encoding='UTF-8', errors='ignore') as fr: 
                txt = fr.read().encode(  'utf-8', errors='ignore' ).decode('utf-8')
                txt=txt.replace('{classname}',PageName) 
                txt=txt.replace('{PK_key}',f"{p['PK_QuestionGroup']}")
                txt=txt.replace('{GroupName}',f"{p['GroupName']}")  
                if ext=='.aspx': 
                    df, code = generate_code_from_db(ctx, qgroup=p['PK_QuestionGroup']) 
                    txt=txt.replace(f'<!--controls-->',code) 
                out=out+txt
            path=f"{dest}{p['PNAME']}{ext}"
            print(path)
            if commit:  
                with open(path, 'w', encoding='UTF-8') as fw:
                    fw.write(txt)
    with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
        fw.write(out)
 
dest=rf"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\FismaForms\\2023\\"  
generate_pages(PK_FORM='2023-A-IG', where=' PK_QuestionGroup < 4461 ', commit=True, PageNameFilter='Function',dest=dest)

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql')
print(code)

In [ ]:
df, code = generate_code_from_db(ctx, 3271)
df[15:25]
#print(code)

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 

In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]